In [ ]:
#https://data-flair.training/blogs/python-deep-learning-project-handwritten-digit-recognition/
#https://pyimagesearch.com/2018/12/31/keras-conv2d-and-convolutional-layers/

import keras
from keras.datasets import mnist #The Keras library already contains some datasets and MNIST is one of them
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

# the data, split between train and test sets
# mnist.load_data() method returns us the training data, its labels and also the testing data and its labels.
#  mnist.load_data() return Tuple of NumPy arrays: (x_train, y_train), (x_test, y_test).
#x_train: uint8 NumPy array of grayscale image data with shapes (60000, 28, 28), containing the training data. Pixel values range from 0 to 255.
#y_train: uint8 NumPy array of digit labels (integers in range 0-9) with shape (60000,) for the training data.
#x_test: uint8 NumPy array of grayscale image data with shapes (10000, 28, 28), containing the test data. Pixel values range from 0 to 255.
#y_test: uint8 NumPy array of digit labels (integers in range 0-9) with shape (10000,) for the test data.


#x-pictures
#y-lables

#x_train.shape == (60000, 28, 28)
#x_test.shape == (10000, 28, 28)
#y_train.shape == (60000,)
#y_test.shape == (10000,)


(x_train, y_train), (x_test, y_test) = mnist.load_data() 

print(x_train.shape, y_train.shape)


#The dimension of the training data is (60000,28,28). The CNN model will 
#require one more dimension (dor the number recognized) so we reshape the matrix to shape (60000,28,28,1).
#x_train.shape is (60000, 28, 28). So x_train.shape[0] is 60000
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

# convert class vectors to binary class matrices
#tf.keras.utils.to_categorical( y, num_classes=None)
#y = Array-like with class values to be converted into a matrix
x_test

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)



# astype - Cast a pandas object to a specified dtype.
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#  /=    Divides the variable by a value and assigns the result to that variable.

x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

batch_size = 128
num_classes = 10
epochs = 10

#A Sequential model is appropriate for a plain stack of 
#layers where each layer has exactly one input tensor and one output tensor.
model = Sequential()
#Conv2D - This layer creates a convolution kernel that is convolved with the layer input to produce a tensor of outputs. 
#kernel_size - An integer or tuple/list of 2 integers, specifying the height and width of the 2D convolution window. Can be a single integer to specify the same value for all spatial dimensions.
#activation - Activation function to use. If you don't specify anything, no activation is applied
#The ReLU is the most used activation function in the world right now.Since, it is used in almost all the convolutional neural networks or deep learning.
model.add(Conv2D(32, kernel_size=(5, 5),activation='relu',input_shape=input_shape))
#Max pooling operation for 2D spatial data.
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
#model.compile  - Configures the model for training.
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])
#model.fit -  Trains the model for a fixed number of epochs (iterations on a dataset).
hist = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))
print("The model has successfully trained")
#model.evaluate - Returns the loss value & metrics values for the model in test mode.
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

model.save('mnist.h5')
print("Saving the model as mnist.h5")

(60000, 28, 28) (60000,)
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Epoch 1/10
469/469 [==============================] - 125s 267ms/step - loss: 2.3083 - accuracy: 0.0975 - val_loss: 2.2944 - val_accuracy: 0.1071
Epoch 2/10
469/469 [==============================] - 126s 268ms/step - loss: 2.2912 - accuracy: 0.1151 - val_loss: 2.2765 - val_accuracy: 0.1280
Epoch 3/10
349/469 [=====================>........] - ETA: 29s - loss: 2.2771 - accuracy: 0.1358

In [ ]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np
model = load_model('mnist.h5')
def predict_digit(img):
    #resize image to 28x28 pixels
    img = img.resize((28,28))
    #convert rgb to grayscale
    img = img.convert('L')
    img = np.array(img)
    #reshaping to support our model input and normalizing
    img = img.reshape(1,28,28,1)
    img = img/255.0
    #predicting the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)
class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.x = self.y = 0
        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="cross")
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command =         self.classify_handwriting) 
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)
        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)
    def clear_all(self):
        self.canvas.delete("all")
    def classify_handwriting(self):
        HWND = self.canvas.winfo_id() # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND) # get the coordinate of the canvas
        im = ImageGrab.grab(rect)
        digit, acc = predict_digit(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')
    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')
app = App()
mainloop()